<a href="https://colab.research.google.com/github/DavidF2714/TC3006B/blob/main/Actividad2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p> <img src=https://cdn-3.expansion.mx/dims4/default/bed6721/2147483647/strip/true/crop/664x400+0+0/resize/780x470!/format/webp/quality/60/?url=https%3A%2F%2Fcdn-3.expansion.mx%2Fmedia%2F2014%2F08%2F11%2Funi6.jpg width=500 height=300>
<p>

# Actividad 1
### Alumno: David Flores Becerril
### Matrícula: A01368391
### Inteligencia artificial avanzada para la ciencia de datos I
### Profesor: Esteban Castillo Juarez
### Grupo 101


## ¿Regresión Logística?

La regresión logística es un modelo estadístico utilizado para predecir la probabilidad de que un evento ocurra. Es especialmente útil cuando el resultado que queremos predecir es una variable categórica, generalmente binaria, es decir, que tiene dos posibles resultados (por ejemplo, sí/no, éxito/fracaso).

En lugar de predecir un valor numérico, como en la regresión lineal, la regresión logística estima la probabilidad de que una observación pertenezca a una de las dos categorías. Utiliza una función logística (o sigmoidea) para transformar la salida del modelo en una probabilidad que va de 0 a 1.

Para esta actividad, se tiene como objetivo el reforzar la teoria de regresión logística aplicando el algoritmo manual y de scikit learn entrando una nueva versión del clasificador utilizando un conjunto de datos de "cáncer de mama"


## Experimentación

Para la realización de este proyecto, se utilizan librerias para el manejo de datos, operaciones matemáticas y de visualización como plotly express

In [1]:
import codecs
import math
import random
import sklearn.linear_model as LogisticRegression # type: ignore
import sklearn.metrics as accuracy_score # type: ignore
from sklearn.metrics import confusion_matrix # type: ignore
import plotly.express as px # type: ignore
import pandas as pd
import numpy as np
import time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Dados los archivos .txt de trainig y testing previamente prepresocedados a partir del conjunto de datos originales, extraeremos los features y labels para cada archivo y se adaptan los datos para su mejor manejo. Las etituetas pasan de ser strings a boolean (1 y 0).

In [2]:
# Function to read data from the file
def read_data(file):
    features = []
    labels = []
    with codecs.open(file, 'r', 'utf-8') as f:
        for line in f:
            data = line.strip().split(',')
            features.append([float(x) for x in data[:-1]])
            label = data[-1]
            if label == 'benign':
                labels.append(0.0)
            elif label == 'malignant':
                labels.append(1.0)
    return features, labels

# Read data
train_features, train_labels = read_data('/content/drive/My Drive/Colab Notebooks/TC3006B/cancerTraining.txt')
test_features, test_labels = read_data('/content/drive/My Drive/Colab Notebooks/TC3006B/cancerTest.txt')

A continuación se definen las funciones de activación a probar para este proyecto. En donde encontramos la función sigmoidal y la función relu. Cabe destacar que la función sigmoidal es la mejor para regresión lógistica, el uso de la función ReLu es para demostrar cómo esta función se adaptaría al entorno propuesto por la regresión lógistica. Para la funcion sigmoidal es importante, limitar los valores que puede dar para evitar valores infinitos.

In [3]:
# Variables
iterations = 150
featureNumber = 9

# Sigmoidal function with numerical stability
def sigmoidal(z):
    z = np.clip(z, -500, 500)
    return 1.0 / (1.0 + math.exp(-z))

# ReLu function
def relu(z):
    return max(0, z)


Para la regresión logística implemnetada de manera manual, se hace uso de tres funciones principales: **gradient** que calcula el gradiente de un conjunto de muestras (sampleList) y pesos (weights) utilizando una función de activación proporcionada (activationFunction). La función **stochasticGradientAscent** implementa el algoritmo de ascenso de gradiente estocástico para ajustar los pesos de un modelo de aprendizaje automático. La función **classifyList** clasifica una lista de datos de prueba utilizando un conjunto de pesos y una función de activación. Para encontrar la mejor alpha y accuracy, se itero 150 veces para cada alhpa con valor de 0.01 hasta 0.30

In [4]:
# Function to calculate gradient
def gradient(sampleList, weights, activationFunction):
    sumElements = 0.0
    for x,y in zip(sampleList, weights):
        sumElements += x*y
    return activationFunction(sumElements)

# Function to calculate the stochastic gradient ascent
def stochasticGradientAscent(trainingList, trainingLabels, featureNumber, activation_func, iterations):
    sampleNumber = len(trainingList)

    for alpha in [i * 0.01 for i in range(1, 31)]:
        weights = [1.0] * featureNumber
        for x in range(iterations):
            sampleIndex = list(range(sampleNumber))
            for y in range(sampleNumber):
                learning_rate = 4 / (1.0 + x + y) + alpha
                randIndex = int(random.uniform(0, len(sampleIndex)))
                sampleGradient = gradient(trainingList[randIndex], weights, activation_func)
                error = trainingLabels[randIndex] - sampleGradient
                temp = []
                for index in range(featureNumber):
                    temp.append(learning_rate * (error * trainingList[randIndex][index]))
                for z in range(featureNumber):
                    weights[z] += temp[z]
                del sampleIndex[randIndex]

    return weights

# Function to classify data
def classifyList(testList, weights, activation_func):
    sumElements = 0.0
    for x, y in zip(testList, weights):
        sumElements += x * y
    probability = activation_func(sumElements)
    if probability > 0.5:
        return 1.0
    else:
        return 0.0

La función classifyLRSciKit entrena un modelo de regresión logística utilizando la biblioteca scikit-learn, evalúa su rendimiento y visualiza la matriz de confusión. Si bien, para sckit-learn le es imposible cambiar el valor de alpha y la función de activación explicitamente. Para el valor alpha, se decidió variar el valor de C, que es cuánto se penaliza el modelo por ajustar demasiado los datos de entrenamiento.

In [5]:
def classifyLRSciKit(train_features, train_labels,test_features,test_labels):
    time_scikit = []
    coefficients = []
    for alpha in np.arange(0.01, 0.3, 0.01):
        start_time = time.time()
        logistic = LogisticRegression.LogisticRegression(max_iter=150, solver='liblinear', penalty='l1', C=alpha)
        logistic.fit(train_features, train_labels)
        predictions = logistic.predict(test_features)

        end_time = time.time()
        time_scikit.append(end_time - start_time)
        coefficients.append(logistic.coef_)

    scikit_predictions = predictions
    scikit_coefficients = coefficients[-1]

    recall = accuracy_score.recall_score(test_labels, predictions)
    accuracy = accuracy_score.accuracy_score(test_labels, predictions)


    cm = confusion_matrix(test_labels, predictions)
    # Visualize the confusion matrix using Plotly Express
    fig = px.imshow(cm,
                    labels=dict(x="Etiquetas precedidas", y="Etiquetas verdaderas", color="Conteo"),
                    x=["Clase 0", "Clase 1"], # Replace with class names if necessary
                    y=["Clase 0", "Clase 1"],
                    title="Matriz de confusión Scikit Learn",
                    color_continuous_scale="Reds")

    fig.update_xaxes(side="bottom")
    fig.show()


    print("Recall score of Scikit Learn:", recall)
    print("Precision score of Scikit Learn:", accuracy)

    return time_scikit, scikit_coefficients, scikit_predictions

## Resultados

Finalmente se inicializan las variables que alojaran los datos para las representaciones visuales de los modelos propuestos

In [6]:
time_scikit, scikit_coefficients, scikit_predictions = classifyLRSciKit(train_features, train_labels, test_features, test_labels)

# Example usage
start_time_sigmoidal = time.time()
weights_sigmoidal = stochasticGradientAscent(train_features, train_labels, featureNumber, sigmoidal, iterations)
end_time_sigmoidal = time.time()

start_time_relu = time.time()
weights_relu = stochasticGradientAscent(train_features, train_labels, featureNumber, relu, iterations)
ebd_time_relu = time.time()

time_sigmoidal = end_time_sigmoidal - start_time_sigmoidal
time_relu = ebd_time_relu - start_time_relu

TotalCorrectSig = 0
TotalPredictionsSig = 0
TotalCorrectRelu = 0
TotalPredictionsRelu = 0

# check the accuracy of the model
for x,y in zip(test_features, test_labels):
    TotalPredictionsSig += 1
    predicted=classifyList(x, weights_sigmoidal, sigmoidal)
    if predicted == y:
        TotalCorrectSig += 1


for x,y in zip(test_features, test_labels):
    TotalPredictionsRelu += 1
    predicted=classifyList(x, weights_relu, relu)
    if predicted == y:
        TotalCorrectRelu += 1

print("Accuracy of the sigmoidal model: ", TotalCorrectSig/TotalPredictionsSig)
print("Accuracy of the ReLu model: ", TotalCorrectRelu/TotalPredictionsRelu)


# Generate predictions for the test set using the manual implementation (e.g., with Sigmoidal activation)
manual_predictions_sigmoidal = [classifyList(sample, weights_sigmoidal, sigmoidal) for sample in test_features]

# Generate predictions for the test set using the manual implementation with ReLU activation
manual_predictions_relu = [classifyList(sample, weights_relu, relu) for sample in test_features]

# #recall and precision
recall_sigmoidal = accuracy_score.recall_score(test_labels, manual_predictions_sigmoidal)
# recall_relu = accuracy_score.precision_score(test_labels, manual_predictions_relu)

print("Recall score of Sigmoidal activation:", recall_sigmoidal)
# print("Recall score of ReLU activation:", recall_relu)

# Generate the confusion matrix for Sigmoidal activation
cm_sigmoidal = confusion_matrix(test_labels, manual_predictions_sigmoidal)

# Visualize the confusion matrix for Sigmoidal activation using Plotly Express
fig_sigmoidal = px.imshow(cm_sigmoidal,
    labels=dict(x="Predicted Labels", y="True Labels", color="Count"),
    x=["Clase 0", "Clase 1"],
    y=["Clase 0", "Clase 1"],
    title="Confusion Matrix - Sigmoidal Activation",
    color_continuous_scale="Blues")
fig_sigmoidal.update_xaxes(side="bottom")
fig_sigmoidal.show()

# Generate the confusion matrix for ReLU activation
cm_relu = confusion_matrix(test_labels, manual_predictions_relu)

# Visualize the confusion matrix for ReLU activation using Plotly Express
fig_relu = px.imshow(cm_relu,
    labels=dict(x="Predicted Labels", y="True Labels", color="Count"),
    x=["Clase 0", "Clase 1"],
    y=["Clase 0", "Clase 1"],
    title="Confusion Matrix - ReLU Activation",
    color_continuous_scale="Greens")
fig_relu.update_xaxes(side="bottom")
fig_relu.show()

data = {
    "Model": ["Sigmoidal", "ReLU", "Scikit Learn"],
    "Execution Time": [time_sigmoidal, time_relu, np.mean(time_scikit)],
}

df = pd.DataFrame(data)

fig = px.bar(df, x="Model", y="Execution Time", title="Execution Time of the Models", labels={"Execution Time": "Time (s)", "Model": "Model"}, color="Model")

fig.show()

df_predictions = pd.DataFrame({
    'Sigmoidal Predictions': manual_predictions_sigmoidal,
    'ReLU Predictions': manual_predictions_relu,
    'Scikit-Learn Predictions': scikit_predictions
})

fig_histogram = px.histogram(df_predictions, barmode='overlay', nbins=20,
                             title="Histogram of Predictions for Each Model",
                             labels={'value': 'Predictions', 'variable': 'Model'},
                             opacity=0.75)
fig_histogram.show()

for i in range(featureNumber):
    feature_values = [x[i] for x in test_features]

    df_scatter = pd.DataFrame({
        'Feature': feature_values,
        'Sigmoidal Predictions': manual_predictions_sigmoidal,
        'ReLU Predictions': manual_predictions_relu,
        'Scikit-Learn Predictions': scikit_predictions
    })

    fig_scatter = px.scatter(df_scatter, x='Feature', y=['Sigmoidal Predictions', 'ReLU Predictions', 'Scikit-Learn Predictions'],
                             title=f'Scatter Plot of Feature {i+1} vs Predictions',
                             labels={'Feature': f'Feature {i+1}', 'value': 'Prediction', 'variable': 'Model'})
    fig_scatter.show()

Recall score of Scikit Learn: 0.9230769230769231
Precision score of Scikit Learn: 0.93
Accuracy of the sigmoidal model:  0.8
Accuracy of the ReLu model:  0.74
Recall score of Sigmoidal activation: 0.2692307692307692


Al finalizar el proyecto se determino que en efecto, el tratar de utilizar una función de activación diferente que la sigmoidal para una regresión logística entorpece brutalmente el modelo de aprendizaje máquina, dada la naturaleza booleana de clasificación. ReLu por su parte puede arrojar múltiples valoresm no sólo booleano.

A la hora de tratar de variar el alpha para el modelo de sci kit learn se determino que cambiar C podría hacer que el modelo sea más o menos complejo, lo que podría requerir ajustes en alpha para asegurar que la optimización sea efectiva (por ejemplo, un modelo muy regularizado puede necesitar un alpha más pequeño para ajustar con precisión los pesos del modelo), pero esto es una consideración secundaria y no una interacción directa.
